In [10]:
import sys
sys.path.append(".")
import unittest
import numpy as np
import matplotlib.pyplot as plt
from susie.timing_data import TimingData
from susie.ephemeris import Ephemeris
from scipy.optimize import curve_fit
from astropy import time

test_epochs = np.array([0, 294, 298, 573])
test_mtts = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
test_mtts_err = np.array([0.00043, 0.00028, 0.00062, 0.00042])
test_tra_or_occ = np.array(['tra','occ','tra','occ'])
test_P_linear =  1.0904734089104249 # period linear
test_P_err_linear = 0.0006807480614626216 # period error linear
test_T0_linear = -0.1010330285087608# conjunction time
test_T0_err_linear = 0.23692091722329203 # conjunction time error

test_P_quad =  1.0892661840315387#period quad
test_P_err_quad =  0.0023688537695923505 # period err quad
test_dPdE = 4.224191878694417e-06#period change by epoch
test_dPdE_err = 7.743357776955459e-06#period change by epoch error
test_T0_quad = -0.000865530018010096  #conjunction time quad
test_T0_err_quad = 0.34674308676945004#conjunction time err quad

test_observed_data = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
test_uncertainties= np.array([0.00043, 0.00028, 0.00062, 0.00042])    


In [6]:
epoch_data = np.array([0, 294, 298, 573])
mid_transit_time_data = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
mid_transit_times_uncertainties_data = np.array([0.00043, 0.00028, 0.00062, 0.00042])
# STEP 2: Break data up into epochs, mid transit times, and error
epochs = epoch_data - np.min(epoch_data)
mid_transit_times = mid_transit_time_data - np.min(mid_transit_time_data)
mid_transit_times_err = mid_transit_times_uncertainties_data
# STEP 2.5 (Optional): Make sure the epochs are integers and not floats
epochs = epochs.astype('int')

In [7]:
transit_times_obj1 = TimingData('jd', epochs, mid_transit_times, mid_transit_times_err, test_tra_or_occ, time_scale='tdb')

In [24]:
ephemeris_obj1 = Ephemeris(transit_times_obj1)

In [25]:
linear_model_data = ephemeris_obj1.get_model_ephemeris('linear')
linear_model_data

{'period': 1.09142234035412,
 'period_err': 4.018260635819698e-07,
 'conjunction_time': -6.728390649943598e-05,
 'conjunction_time_err': 0.00013969441566829874,
 'model_type': 'linear',
 'model_data': array([-6.72839065e-05,  3.20878101e+02,  3.25243790e+02,  6.25384934e+02])}

In [26]:
def linear_model(test_x, test_P_linear, test_T0_linear):
    """Linear model function."""
    return test_P_linear * test_x + test_T0_linear


popt, pcov = curve_fit(linear_model, test_epochs, test_mtts, sigma=test_mtts_err, absolute_sigma=True)
unc = np.sqrt(np.diag(pcov))
return_data = {
    'period': popt[0],
    'period_err': unc[0],
    'conjunction_time': popt[1],
    'conjunction_time_err': unc[1]
}
return_data

{'period': 1.0914223408652188,
 'period_err': 9.998517417992763e-07,
 'conjunction_time': -6.734666196939187e-05,
 'conjunction_time_err': 0.0003502975050463415}

In [27]:
def quad_model(test_x, test_dPdE, test_P_quad, test_T0_quad):
    return 0.5*test_dPdE*test_x*test_x + test_P_quad*test_x + test_T0_quad
popt, pcov = curve_fit(quad_model, test_epochs, test_mtts, sigma=test_mtts_err, absolute_sigma=True)
unc = np.sqrt(np.diag(pcov))
return_data = {
            'conjunction_time': popt[2],
            'conjunction_time_err': unc[2],
            'period': popt[1],
            'period_err': unc[1],
            'period_change_by_epoch': popt[0],
            'period_change_by_epoch_err': unc[0],
        }
return_data

{'conjunction_time': -1.415143555084551e-06,
 'conjunction_time_err': 0.00042940561938685084,
 'period': 1.0914215464474404,
 'period_err': 9.150815726215122e-06,
 'period_change_by_epoch': 2.7744598987630543e-09,
 'period_change_by_epoch_err': 3.188345582283935e-08}

In [28]:
quadratic_model_data = ephemeris_obj1.get_model_ephemeris('quadratic')
quadratic_model_data

{'period': 1.091421537944888,
 'period_err': 1.3369529628386657e-06,
 'conjunction_time': -8.102185298292572e-07,
 'conjunction_time_err': 0.00019627998726801882,
 'period_change_by_epoch': 2.8048128182651046e-09,
 'period_change_by_epoch_err': 4.3654905738095855e-09,
 'model_type': 'quadratic',
 'model_data': array([-8.10218530e-07,  3.20878053e+02,  3.25243742e+02,  6.25385001e+02])}

In [50]:
##linear fit
P = 1.091423
E = test_epochs
T0 = 0
tra_or_occ =  np.array([0,1,0,1])
tra = np.round(P * E +T0)
print(tra)
occ = np.round(P * E +(T0+0.5*P))
print(occ)
lin_fit = np.array([0.         , 3.21423879e+02, 325.24385758, 6.25930712e+02 ])


[  0. 321. 325. 625.]
[  1. 321. 326. 626.]


In [51]:
##quad fit
P = 1.091423
E =  np.array([0, 294, 298, 573])
T0 = 0
dPdE = 0
tra_q = np.round(0.5*dPdE*E*E + P*E + 0)
print(tra_q)
occ_q = np.round((0 + 0.5*P) + P*E + 0.5*dPdE*E*E)
print(occ_q)
quad_fit = np.array([0,321,325,626])

[  0. 321. 325. 625.]
[  1. 321. 326. 626.]


In [9]:
##linear model uncertainites
T0_err_l =  0.23692091722329203
P_err_l =  0.0006807480614626216
E = test_epochs
tra = (np.sqrt((T0_err_l**2) + ((test_epochs**2)*(P_err_l**2))))
print(tra)
occ = (np.sqrt(((T0_err_l**2) +((test_epochs+0.5)**2) * (P_err_l**2))))
print(occ)
result = np.array([0.23692092,0.31036088, 0.31190525, 0.45667354 ])

[0.23692092 0.31014112 0.31190525 0.45638259]
[0.23692116 0.31036088 0.31212674 0.45667354]


In [8]:
##quad model uncertainite
T0_err_q = 0.34674308676945004
P_err_q = 0.0023688537695923505
dPdE_err = 7.743357776955459e-06
E = test_epochs
tra = (np.sqrt((T0_err_q**2) + ((E**2)*(P_err_q**2)) + ((1/4)*(E**4)*(dPdE_err**2))))
print(tra)

occ = (np.sqrt( (T0_err_q**2) + (((E+0.5)**2)*(P_err_q**2)) + ((1/4)*(E**4)*(dPdE_err**2))))
print(occ)
result = np.array([0.34674309,0.84788387,0.85834968,1.89255521])


[0.34674309 0.84690961 0.85834968 1.89170516]
[0.34674511 0.84788387 0.85932403 1.89255521]


In [57]:
##calc lin ephem
E = test_epochs
tra = -0.1010330285087608+1.0904734089104249*E
print(tra)
occ = (-0.1010330285087608+0.5*1.0904734089104249)+1.0904734089104249*E
print(occ)
result = np.array([-1.01033029e-01, 3.21043386e+02 , 3.24860043e+02, 6.25285467e+02 ])

[-1.01033029e-01  3.20498149e+02  3.24860043e+02  6.24740230e+02]
[4.44203676e-01 3.21043386e+02 3.25405280e+02 6.25285467e+02]


In [65]:
## calc quad ephem
P_q = 1.0892661840315387
T0_q =  -0.000865530018010096
dPdE =  4.224191878694417e-06
tra = (T0_q + P_q*E + 0.5*dPdE*E*E)
occ = ((T0_q + 0.5*P_q) + P_q*E + 0.5*dPdE*E*E)
print(tra)
print(occ)
result = np.array([-8.65530018e-04,3.20970587e+02,3.24788020e+02,6.25386753e+02])

[-8.65530018e-04  3.20425954e+02  3.24788020e+02  6.24842120e+02]
[5.43767562e-01 3.20970587e+02 3.25332653e+02 6.25386753e+02]


In [68]:
#calc chi linear
observed_data = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
model_data =  np.array([-1.01033029e-01, 3.21043386e+02 , 3.24860043e+02, 6.25285467e+02 ])
uncertainties = np.array([0.00043, 0.00028, 0.00062, 0.00042]) 
np.sum(((observed_data - model_data)/uncertainties)**2)

843766.361148408

In [69]:
#calc chi quad
observed_data = np.array([0.0, 320.8780000000261, 325.24399999994785, 625.3850000002421])
model_data =  np.array([-8.65530018e-04,3.20970587e+02,3.24788020e+02,6.25386753e+02])
uncertainties =  np.array([0.00043, 0.00028, 0.00062, 0.00042])
np.sum(((observed_data - model_data)/uncertainties)**2)

650251.7787726448

In [70]:
## calc bic lin
chi_squared= 843766.361148408
num_params=2

chi_squared + (num_params*np.log(4))


843769.1337371303

In [71]:
## calc bic quad
chi_squared= 650251.7787726448
num_params=3

chi_squared + (num_params*np.log(4))

650255.9376557282

In [72]:
linear_bic =  843769.1337371303
quadratic_bic =  650255.9376557282
linear_bic - quadratic_bic

193513.19608140213

In [93]:
from astropy.time import Time, TimeDelta

# Example mid_times data
mid_times = ['2024-05-23T12:34:56', '2024-05-24T12:34:56']
time_format = 'isot'  # ISO 8601 format
time_scale = 'utc'    # Coordinated Universal Time scale

mid_times_obj = Time(mid_times, format=time_format, scale=time_scale)
mid_times_obj

<Time object: scale='utc' format='jd' value=[2457585.914587]>

In [94]:
test_mid_times = ['2024-05-23T12:34:56', '2024-05-24T12:34:56']
test_time_format = 'isot'  
test_time_scale = 'utc' 

# Create the Time object
mid_times_obj = Time(mid_times, format=time_format, scale=time_scale)

# Output the Time object and its attributes
print(mid_times_obj)
print("Julian Date:", mid_times_obj.jd)
print("Barycentric Dynamical Time:", mid_times_obj.tdb)

[2457585.914587]
Julian Date: [2457585.914587]
Barycentric Dynamical Time: [2457585.91537616]


In [86]:
test_mid_times_uncertainties = ['0.1', '0.2']
test_time_format = 'jd'  
test_time_scale = 'tdb' 
test_mid_times_uncertainties_obj = TimeDelta(test_mid_times_uncertainties, format=test_time_format, scale=test_time_scale)
print(test_mid_times_uncertainties_obj)
print("Julian Date:", test_mid_times_uncertainties_obj.jd)
print("Barycentric Dynamical Time:", test_mid_times_uncertainties_obj.tdb)

[0.1 0.2]
Julian Date: [0.1 0.2]
Barycentric Dynamical Time: [0.1 0.2]


In [85]:
from astropy.time import TimeDelta

# Example uncertainties data
mid_time_uncertainties = ['0.1', '0.2']  # Uncertainties in seconds
time_format = 'jd'  
time_scale = 'tdb' 

# Convert uncertainties to floating-point numbers and seconds
mid_time_uncertainties = [float(uncertainty) for uncertainty in mid_time_uncertainties]

# Create the TimeDelta object representing uncertainties
mid_time_uncertainties_obj = TimeDelta(mid_time_uncertainties, format=time_format, scale=time_scale)

# Output the TimeDelta object and its attributes
print(mid_time_uncertainties_obj)

[0.1 0.2]


In [5]:
from astropy.time import Time

# Define the time value in BJD-UTC
time_bjd_utc = 2457585.914587

# Create a Time object with the BJD-UTC value
time_obj = Time(time_bjd_utc, format='jd', scale='utc')

# Convert the time value to BJD-TDB
time_bjd_tdb = time_obj.tdb.jd

# Output the converted value
print("BJD-TDB:", time_bjd_tdb)

BJD-TDB: 2457585.9153761626


In [13]:
mid_times = np.array([2457585.914587, 2457586.914587])  # An array of float times
time_format = 'jd'  # Julian Date format
time_scale = 'tdb'  # Coordinated Universal Time scale

# Create the Time object
mid_times_obj = Time(mid_times, format=time_format, scale=time_scale)

# Output the Time object and its attributes
print(mid_times_obj)
# print("Julian Date:", mid_times_obj.jd)
# print("Barycentric Dynamical Time:", mid_times_obj.tdb)

[2457585.914587 2457586.914587]


In [8]:
object_ra = 150.0  # Right ascension in degrees
object_dec = 2.5   # Declination in degrees
obj_coords = (object_ra, object_dec)
print("\nObject coordinates (RA, DEC):")
print(obj_coords)

# Observatory coordinates (Longitude, Latitude)
observatory_lon = -70.0  # Longitude in degrees
observatory_lat = -30.0  # Latitude in degrees
obs_coords = (observatory_lon, observatory_lat)
print("\nObservatory coordinates (Longitude, Latitude):")
print(obs_coords)


Object coordinates (RA, DEC):
(150.0, 2.5)

Observatory coordinates (Longitude, Latitude):
(-70.0, -30.0)


In [11]:
import numpy as np
from astropy.time import Time

# Example mid-transit times (Julian Dates)
mid_times = np.array([2457585.914587, 2457586.914587])

# Create the Time object with Julian Date format and Barycentric Dynamical Time scale
test_mid_times_obj = Time(mid_times, scale='tdb')

# Print the mid-transit times object
print("Mid-transit times (JD, TDB):")
print(test_mid_times_obj)

ValueError: Input values did not match any of the formats where the format keyword is optional:
- 'datetime': Input values for datetime class must be datetime objects
- 'ymdhms': input must be dict or table-like
- 'iso': Input values for iso class must be strings
- 'isot': Input values for isot class must be strings
- 'yday': Input values for yday class must be strings
- 'datetime64': Input values for datetime64 class must be datetime64 objects
- 'fits': Input values for fits class must be strings
- 'byear_str': Input values for byear_str class must be strings
- 'jyear_str': Input values for jyear_str class must be strings
- 'astropy_time': Input values for astropy_time class must all be the same astropy Time type.

In [16]:

time_mtts = time.Time(test_mtts, format='jd', scale='tdb')
time_mtts_err = time.Time(test_mtts_err, format='jd', scale='tdb')  

converted_mtts = time_mtts.to_value( 'utc','isot')
converted_mtts_err = time_mtts_err.to_value( 'utc','isot')

ValueError: format must be one of ['jd', 'mjd', 'decimalyear', 'unix', 'unix_tai', 'cxcsec', 'gps', 'plot_date', 'stardate', 'datetime', 'ymdhms', 'iso', 'isot', 'yday', 'datetime64', 'fits', 'byear', 'jyear', 'byear_str', 'jyear_str']